# TensorRT

In [1]:
import torch
import torchvision
from torch2trt import TRTModule

device = torch.device('cuda')

In [2]:
model = TRTModule()
model.load_state_dict(torch.load('/workspace/banana/best_model_MTL_2_trt.pth'))

<All keys matched successfully>

In [3]:
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    tran = transforms.Compose([
        transforms.Grayscale(3), 
        transforms.ToTensor()
    ]
    )
    image = tran(image).to(device).half()
    
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

In [4]:
from IPython.display import display
import ipywidgets
import traitlets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera()

In [5]:
speed_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, description='speed gain')
add_motor = ipywidgets.FloatSlider(min=0.0, max=2.0, step=0.01, value=0.35, description='add_motor')#add each motor speed
dec_motor = ipywidgets.FloatSlider(min=0.0, max=2.0, step=0.01, value=0.30, description='dec_motor')
diff_slider = ipywidgets.FloatSlider(min=0.1, max=1, step=0.01, value=0.2, description='add_diff')#add diff direction value
pid_value = ipywidgets.FloatSlider(min=0.1, max=1, step=0.1, value=0.4 ,description='pid value')

left_value = ipywidgets.FloatSlider(min=0.0, max=1, step=0.1, value=0.0 ,description='left value')
right_value = ipywidgets.FloatSlider(min=0.0, max=1, step=0.1, value=0.0 ,description='right value')

image_widget = ipywidgets.Image()
draw_widget = ipywidgets.Image(width=camera.width, height=camera.height)

x_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', readout_format='0.3f', description='y')
block_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, orientation='vertical', description='block')
steering_slider = ipywidgets.FloatSlider(min=-3.0, max=3.0, readout_format='0.3f', description='pid')
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, step=0.05, value=0.25, orientation='vertical', description='speed')

controll_box = ipywidgets.VBox([speed_gain_slider, add_motor,dec_motor, diff_slider, pid_value, left_value, right_value])
display(ipywidgets.HBox([ipywidgets.HBox([y_slider, draw_widget]), block_slider, speed_slider, controll_box]))
display(x_slider, steering_slider)

from jetbot import Robot
robot = Robot()

FloatSlider(value=0.0, description='x', max=1.0, min=-1.0)

FloatSlider(value=0.0, description='pid', max=3.0, min=-3.0, readout_format='0.3f')

In [6]:
from BotLine import *

botLine = BotLine('JETBOT', '192.168.0.14', 8000, 10.0)

[2021-11-11 12:56:54.279063] Connecting...


In [7]:
import time

angle_last = 0.0
left = 0.0
right = 0.0
isStop = False

t0 = time.time()

def execute(change):
    global angle_last, left, right, isStop, t0
    image = change['new']
    result = model(preprocess(image))
    xy = result[0].float().cpu().numpy().flatten()
    block = result[1].float().cpu().numpy().flatten()
    
    x = round(xy[0], 3)
    y = round((0.5 - xy[1]), 3)
    
    x_slider.value = x
    y_slider.value = y
    block_slider.value = block
    
    angle = round(np.arctan2(x, y),3) #radian
    
    if (botLine.botLineObject.isStop() is False) and (botLine.botLineObject.isWorking() is True):
        if block_slider.value >= 0.55:
            robot.stop()
            isStop = True
            left = 0
            right = 0
        else:
            if ( (abs(angle - angle_last) > abs(angle_last)) and ( angle_last != 0) and (abs(angle) > pid_value.value)):
                angle = angle_last + ( angle * diff_slider.value )

                pid = round(angle ,2)

                if pid > 0 :
                    left = round(pid * add_motor.value * speed_slider.value + speed_slider.value , 2)
                    right = round(-pid * dec_motor.value * speed_slider.value + speed_slider.value, 2)
                else:
                    left = round(pid * dec_motor.value * speed_slider.value + speed_slider.value , 2)
                    right = round(-pid * add_motor.value * speed_slider.value + speed_slider.value, 2)
            else:
                pid = round(angle ,2) #0.95 etc..
                left = round(max(pid, 0)* add_motor.value * speed_slider.value + speed_slider.value , 2)
                right = round(max(-pid, 0)* add_motor.value * speed_slider.value + speed_slider.value, 2)

                steering_slider.value = pid

                #전역변수 설정
            angle_last = angle
            robot.left_motor.value = left
            robot.right_motor.value = right
    else:
        robot.stop()
    
    image_data = image.copy()
    circle_color = (0, 255, 0)
    if block_slider.value >= 0.55:
        circle_color = (0, 0, 255)
    image_data = cv2.circle(image_data, 
                            (112 + int(x_slider.value * 112), 
                             224 - int(y_slider.value * 224)), 8, circle_color, 3)
    draw_widget.value = bgr8_to_jpeg(image_data)
    
    botLine.onUpdate()
    time.sleep(0.001)
        
execute({'new': camera.value})

[2021-11-11 12:56:55.278334] Connect(Hash: 410429634)


In [8]:
camera.observe(execute, names='value')

In [10]:
import time

camera.unobserve(execute, names='value')

time.sleep(0.1)  # add a small sleep to make sure frames have finished processing
robot.stop()

In [11]:
botLine.onDestory()

In [12]:
camera.stop()